In [27]:
import pandas as pd
import datetime
from dateutil import relativedelta
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("Nat_Gas.csv")
DatesObj = []
months = []
years = []
seasons = []
for i in range(data.shape[0]):
    date_str = data["Dates"].iloc[i]
    month_day_yr = date_str.split("/")
    month_day_yr_int = [int(x) for x in month_day_yr]
    dateobj = datetime.date(month_day_yr_int[2],month_day_yr_int[0],month_day_yr_int[1])
    if dateobj.month >= 12 or dateobj.month < 3:
        seasons.append(1)
    elif dateobj.month >= 3 and dateobj.month < 6:
        seasons.append(2)
    elif dateobj.month >= 6 and dateobj.month < 9:
        seasons.append(3)
    elif dateobj.month >= 9 and dateobj.month < 12:
        seasons.append(4)
    DatesObj.append(dateobj)
    months.append(dateobj.month)
    years.append(dateobj.year)
data["DatesAsObj"] = DatesObj
data["month"] = months
data["year"] = years
data["season"] = seasons
data.drop("Dates", axis = 1, inplace=True)
data.drop("DatesAsObj", axis=1, inplace=True)

In [3]:
X = data[["month", "year", "season"]]
y = data["Prices"]

In [4]:
X_train, y_train = X[:35], y[:35]
X_val, y_val = X[35:40], y[35:40]
X_test, y_test = X[40:], y[40:]

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

cp = ModelCheckpoint("NatGasModel", save_best_only=True)
model = Sequential([layers.Input((3,1), name="input"),
                    layers.LSTM(128),
                    layers.Dense(64),
                    layers.Dense(1, name = "Dense_layer"),
                    layers.Activation("linear", name = "output")])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5000, callbacks = cp)

ImportError: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (C:\Users\edgar\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\backend\__init__.py)

In [ ]:
model.predict(X_val)

In [ ]:
print(X_val)

In [5]:
def PredictGivenDate(date):
    datesplit = date.split("/")
    datesplit = [int(x) for x in datesplit]
    if dateobj.month >= 12 or dateobj.month < 3:
        seasonpred = 1
    elif dateobj.month >= 3 and dateobj.month < 6:
        seasonpred = 2
    elif dateobj.month >= 6 and dateobj.month < 9:
        seasonpred = 3
    elif dateobj.month >= 9 and dateobj.month < 12:
        seasonpred = 4
    array2pred = np.array([datesplit[1], datesplit[0], seasonpred])
    array2pred = np.resize(array2pred,(array2pred.shape[0],1))
    from keras.models import load_model
    filename = "NatGasModel"
    model = load_model(filename)
    return model.predict(np.reshape(array2pred, (1,3)))[0][0]

In [6]:
date = input("Please input date in format of (2-digit year/2-digit month/2-digit day)")
pred = PredictGivenDate(date)
print(pred)


KeyboardInterrupt



In [7]:
def GetInjectionsDateAndQ():
    Injections = {"Date" : [], "Quantity" : []}
    while True:
        date = input("Please input an injection date in format of (4-digit year/2-digit month/2-digit day)")
        yr_month_day = date.split("/")
        yr_month_day_int = [int(x) for x in yr_month_day]
        dateobj = datetime.date(yr_month_day_int[0],yr_month_day_int[1],yr_month_day_int[2])
        quantity = int(input("Please input the quantity of gas you will purchase at this date, e.g. 1000000: "))
        Injections["Date"].append(dateobj)
        Injections["Quantity"].append(quantity)
        quit = input("Would you like to enter more injections? y/n:")
        if quit == "y":
            continue
        else:
            break
    InjectionsPd = pd.DataFrame(data=Injections, columns=["Date", "Quantity"])
    return InjectionsPd

In [8]:
def GetWithdrawalsDateAndQ():
    Withdrawals = {"Date" : [], "Quantity" : []}
    while True:
        date = input("Please input a withdrawal date in format of (4-digit year/2-digit month/2-digit day)")
        yr_month_day = date.split("/")
        yr_month_day_int = [int(x) for x in yr_month_day]
        dateobj = datetime.date(yr_month_day_int[0],yr_month_day_int[1],yr_month_day_int[2])
        quantity = int(input("Please input the quantity of gas you will sell at this date, e.g. 1000000: "))
        Withdrawals["Date"].append(dateobj)
        Withdrawals["Quantity"].append(quantity)
        quit = input("Would you like to enter more withdrawals? y/n:")
        if quit == "y":
            continue
        else:
            break
    WithdrawalsPd = pd.DataFrame(data=Withdrawals, columns=["Date", "Quantity"])
    return WithdrawalsPd

In [9]:
def UpdateInjections(Injections):
    prices = []
    pricesbyq = []
    type = []
    from keras.models import load_model
    filename = "NatGasModel"
    model = load_model(filename)
    for i in range(len(Injections)):
        seasonpred = 0
        month = Injections["Date"].iloc[i].month
        year = Injections["Date"].iloc[i].year
        if Injections["Date"].iloc[i].month >= 12 or Injections["Date"].iloc[i].month < 3:
            seasonpred = 1
        elif Injections["Date"].iloc[i].month >= 3 and Injections["Date"].iloc[i].month < 6:
            seasonpred = 2
        elif Injections["Date"].iloc[i].month >= 6 and Injections["Date"].iloc[i].month < 9:
            seasonpred = 3
        elif Injections["Date"].iloc[i].month >= 9 and Injections["Date"].iloc[i].month < 12:
            seasonpred = 4
        array2pred = np.array([month, year, seasonpred])
        array2pred = np.resize(array2pred,(array2pred.shape[0],1))
        price = model.predict(np.reshape(array2pred, (1,3)))[0][0]
        prices.append(price)
        type.append("I")
        pricesbyq.append(price * int(Injections["Quantity"].iloc[i]))
    Injections["PriceAtDate"] = prices
    Injections["PriceByQ"] = pricesbyq
    Injections["Type"] = type
    return Injections

In [10]:
def UpdateWithdrawals(Withdrawals):
    prices = []
    pricesbyq = []
    type = []
    from keras.models import load_model
    filename = "NatGasModel"
    model = load_model(filename)
    for i in range(len(Withdrawals)):
        seasonpred = 0
        month = Withdrawals["Date"].iloc[i].month
        year = Withdrawals["Date"].iloc[i].year
        if Withdrawals["Date"].iloc[i].month >= 12 or Withdrawals["Date"].iloc[i].month < 3:
            seasonpred = 1
        elif Withdrawals["Date"].iloc[i].month >= 3 and Withdrawals["Date"].iloc[i].month < 6:
            seasonpred = 2
        elif Withdrawals["Date"].iloc[i].month >= 6 and Withdrawals["Date"].iloc[i].month < 9:
            seasonpred = 3
        elif Withdrawals["Date"].iloc[i].month >= 9 and Withdrawals["Date"].iloc[i].month < 12:
            seasonpred = 4
        array2pred = np.array([month, year, seasonpred])
        array2pred = np.resize(array2pred,(array2pred.shape[0],1))
        price = model.predict(np.reshape(array2pred, (1,3)))[0][0]
        prices.append(price)
        type.append("W")
        pricesbyq.append(price * int(Withdrawals["Quantity"].iloc[i]))
    Withdrawals["PriceAtDate"] = prices
    Withdrawals["PriceByQ"] = pricesbyq
    Withdrawals["Type"] = type
    
    return Withdrawals

In [32]:
def TypePlusMinusPrice(df, runningtotal, rate):
    for i in range(len(df)):
        if df.iloc[i]["Type"] == "I":
            runningtotal -= df.iloc[i]["PriceByQ"]
            runningtotal -= rate
        elif df.iloc[i]["Type"] == "W":
            runningtotal += df.iloc[i]["PriceByQ"]
            runningtotal -= rate
    return runningtotal

In [62]:
def TypePlusMinusQuantity(df, runningquantity, maxvolume):
    for i in range(len(df)):
        if 0 > runningquantity > maxvolume:
            print(f"At date: {df.iloc[i]['Date']} the stored valume is greater than maximum.")
        elif df.iloc[i]["Type"] == "I":
            runningquantity += df.iloc[i]["Quantity"]
        elif df.iloc[i]["Type"] == "W":
            runningquantity -= df.iloc[i]["Quantity"]
    return runningquantity

In [34]:
def RunningQuantity(df):
    runningquantity = 0
    maxvolume = float(input("Enter the maximum volume which can be stored at one time: "))
    return TypePlusMinusQuantity(df, runningquantity, maxvolume)

In [45]:
def RunningTotal(df):
    runningtotal = 0
    firstdate = df.iloc[0]["Date"]
    lastdate = df.iloc[-1]["Date"]
    rate = float(input("Enter the rate for an injection/withdrawal: "))
    runningtotal = TypePlusMinusPrice(df, runningtotal, rate)
    return StorageCosts(runningtotal, firstdate, lastdate)

In [55]:
def DiffBetweenTwoDates(date1, date2):
    r = relativedelta.relativedelta(date1, date2)
    return (r.years * 12) + r.months

In [58]:
def StorageCosts(runningtotal, date1, date2):
    storagecostspm = float(input("Please enter the storage costs per month: "))
    dist = float(DiffBetweenTwoDates(date1, date2))
    return runningtotal + (storagecostspm * dist)

In [48]:
def PriceContract():
    Injections = GetInjectionsDateAndQ()
    Withdrawals = GetWithdrawalsDateAndQ()
    Injections = UpdateInjections(Injections)
    Withdrawals = UpdateWithdrawals(Withdrawals)
    df = pd.concat([Injections, Withdrawals]).sort_values("Date")
    runningtotal = RunningTotal(df)
    runningquantity = RunningQuantity(df)
    
    return df, runningtotal, runningquantity

In [63]:
df, rt, rq = PriceContract()

Please input an injection date in format of (4-digit year/2-digit month/2-digit day) 2021/12/20
Please input the quantity of gas you will purchase at this date, e.g. 1000000:  100
Would you like to enter more injections? y/n: y
Please input an injection date in format of (4-digit year/2-digit month/2-digit day) 2023/12/20
Please input the quantity of gas you will purchase at this date, e.g. 1000000:  1000
Would you like to enter more injections? y/n: n
Please input a withdrawal date in format of (4-digit year/2-digit month/2-digit day) 2022/12/20
Please input the quantity of gas you will sell at this date, e.g. 1000000:  100
Would you like to enter more withdrawals? y/n: n


1/1 [==============================] - 1s 1s/step


Enter the rate for an injection/withdrawal:  1000
Please enter the storage costs per month:  1000
Enter the maximum volume which can be stored at one time:  10000


In [64]:
rt

-47040.51208496094

In [65]:
rq

1000